# Import Libraries and Functions

In [ ]:
import pandas as pd
import polars as pl
import requests
import numpy as np
import re
import os
import time
from datetime import datetime, timedelta
from functools import reduce
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [ ]:
def gcp2df(sql): #BigQuery to Pandas DataFrame
  query = client.query(sql)
  results = query.result()
  return results.to_dataframe()

def no_accent_plus(s):
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[ÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬ]', 'A', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[ÉÈẺẼẸÊẾỀỂỄỆ]', 'E', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[ÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢ]', 'O', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[ÍÌỈĨỊ]', 'I', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ÚÙỦŨỤƯỨỪỬỮỰ]', 'U', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('[ÝỲỶỸỴ]', 'Y', s)
    s = re.sub('đ', 'd', s)
    s = re.sub('Đ', 'D', s)
    s = re.sub("\s\s+", " ", s)
    s = re.sub("\([^)]*\)", "", s) #Remove parentheses
    s = s.lower().strip()
    return s

def WeekdayCalculation(d, weekday = 1, status = 'prev'):
    d = datetime.strptime(d, "%Y-%m-%d")
    if status == 'prev':
        interval = d.weekday() - weekday # number represents weekday: 0 for Monday
        value_date = d - timedelta(days = interval)
    if status == 'next':
        interval = weekday - d.weekday()  # number represents weekday: 0 for Monday
        interval = interval + 7 if interval < 0 else interval
        value_date = d + timedelta(days = interval)
    return value_date.strftime('%Y-%m-%d')

def WeekRangeInput(inp):
    day_num = 5 # Insert end of week day_num (Monday = 0,...)
    def StartWeekOfYear(year):
        date_start = datetime(year, 1, 1)  # January 1st of the specified year
        while date_start.weekday() != day_num:
            date_start += timedelta(days=1)
        date_end = date_start + timedelta(weeks = 51)
        return [date_start, date_end]

    def NextDayCalculation(d):
        days_ahead = day_num - d.weekday()
        days_ahead = days_ahead + 7 if days_ahead < 0 else days_ahead
        value_date = d + timedelta(days_ahead)
        return value_date

    x = datetime.strptime(inp, "%Y-%m-%d")
    year_value = x.year if x <= StartWeekOfYear(x.year)[1] else x.year + 1
    first_start_day = StartWeekOfYear(year_value)[0]

    end_day = NextDayCalculation(x)
    start_day = end_day - timedelta(days = 6)

    end_day_string = end_day.strftime("%m/%d")
    start_day_string = start_day.strftime("%m/%d")

    week_num = int((end_day - first_start_day).days/7 + 1)
    week_range = "Tuần " + str(year_value) + '/' + str(week_num).zfill(2) +': ' + start_day_string + '-' + end_day_string
    return week_range

In [ ]:
department_list = ['hno', 'dsh', 'bd north', 'bd south']

# **Bảng 1 + Bảng 2**

In [ ]:
wb_so = gc.open_by_url('https://docs.google.com/spreadsheets/d/15wjltkhNcDL07oFkAadpazZQg-O7AXx0QpD_z0_6Grk/edit#gid=1568281000')
ws_so = wb_so.worksheet('ChiTiet_level_1')
data = pd.DataFrame(ws_so.get_all_records())

**QUICK NOTE:**
* Bốn câu được dùng để Tạo bảng Level 1:
  * CODING Tình trạng hiện tại (1)
  * CODING Nguyên nhân giảm đơn / rời bỏ chính nhất (2)
  * CODING Share đơn (3)
  * CODING Share đơn trên 50% (4)
* WeekFinal sẽ được lùi về ngày Thứ 3 của Tuần không kể đợt là Thứ 3 hay Thứ 6
* Cột id_col được dùng kết hợp với 2 câu coding nêu trên với mục đích drop duplicates: Bỏ qua đợt là thứ 3 hay thứ 6.

* Percentage được tính theo phần trăm của từng nhóm
(VD: nhóm Cá nhân của giảm đơn thì sẽ tính bằng Lý do cá nhân/Tổng của nhóm Giảm đơn)


In [ ]:
coding_cols = [ 'Dot_V2', 'clientid', 'department_name',
                'CODING Tình trạng hiện tại', 'CODING Nguyên nhân giảm đơn / rời bỏ chính nhất',
                '1.1. Vận hành_CODING_SO'
               ]

df_so_raw = data[coding_cols]

In [ ]:
#Data Manipulation
df_so_raw['Dot_V2'] = pd.to_datetime(df_so_raw['Dot_V2'], errors = 'coerce')
df_so_raw['THANG'] = df_so_raw['Dot_V2'].to_numpy().astype('datetime64[M]')
df_so_raw['THANG'] = df_so_raw['THANG'].dt.date
df_so_raw['clientid'] = df_so_raw['clientid'].astype(str)

In [ ]:
df_so_raw = df_so_raw[~df_so_raw['Dot_V2'].isnull()]
df_so_raw['WeekFinal'] = df_so_raw['Dot_V2'].dt.strftime('%Y-%m-%d').apply(WeekdayCalculation)
df_so_raw['WeekFinal_2'] = df_so_raw['Dot_V2'].dt.strftime('%Y-%m-%d').apply(WeekRangeInput)

df_so_raw['id_col'] = df_so_raw['clientid'] + '_' +df_so_raw['WeekFinal'] #Dùng để drop Duplicates

In [ ]:
df_so_raw['department_name'] = df_so_raw['department_name'].astype(str).apply(no_accent_plus)
df_so_raw['department_name_2'] = np.where(df_so_raw['department_name'].isin(department_list), df_so_raw['department_name'], 'others')

In [ ]:
df_so_raw['CODING Tình trạng hiện tại'] = df_so_raw['CODING Tình trạng hiện tại'].astype(str).apply(no_accent_plus)
df_so_raw['CODING Nguyên nhân giảm đơn / rời bỏ chính nhất'] = df_so_raw['CODING Nguyên nhân giảm đơn / rời bỏ chính nhất'].astype(str).apply(no_accent_plus)
df_so_raw['1.1. Vận hành_CODING_SO'] = df_so_raw['1.1. Vận hành_CODING_SO'].astype(str).apply(no_accent_plus)

In [ ]:
df_so_raw = df_so_raw.replace(r'^\s*$', np.nan, regex=True)
df_so_raw = df_so_raw.sort_values(by = ['id_col',
                                        'CODING Tình trạng hiện tại', 'CODING Nguyên nhân giảm đơn / rời bỏ chính nhất', '1.1. Vận hành_CODING_SO',
                                        'Dot_V2'],
                                        ascending = [True, False, False, False, True], na_position = 'last')
df_so_raw

In [ ]:
df_so = df_so_raw.drop_duplicates(subset = ['id_col'], keep = 'first')

In [ ]:
df_so['is_giamdon'] = np.where(df_so['CODING Tình trạng hiện tại'].str.contains(r'giam don', na = False), 1, 0)
df_so['is_roibo'] = np.where(df_so['CODING Tình trạng hiện tại'].str.contains(r'roi bo', na = False), 1, 0)

df_so['LV1_CaNhan'] = np.where(df_so['CODING Nguyên nhân giảm đơn / rời bỏ chính nhất'].str.contains(r'ca nhan', na = False), 1, 0)
df_so['LV1_VanHanh'] = np.where(df_so['CODING Nguyên nhân giảm đơn / rời bỏ chính nhất'].str.contains(r'van hanh', na = False), 1, 0)
df_so['LV1_DenBu'] = np.where(df_so['CODING Nguyên nhân giảm đơn / rời bỏ chính nhất'].str.contains(r'den bu', na = False), 1, 0)
df_so['LV1_Gia'] = np.where(df_so['CODING Nguyên nhân giảm đơn / rời bỏ chính nhất'].str.contains(r'gia|phu phi', na = False), 1, 0)
df_so['LV1_CSKH'] = np.where(df_so['CODING Nguyên nhân giảm đơn / rời bỏ chính nhất'].str.contains(r'cham soc khach hang|cskh', na = False), 1, 0)
df_so['LV1_Khac'] = np.where(df_so['CODING Nguyên nhân giảm đơn / rời bỏ chính nhất'].str.contains(r'khac', na = False), 1, 0)


df_so['LV2_TatTuyen'] = np.where(df_so['1.1. Vận hành_CODING_SO'].str.contains(r'tat tuyen', na = False), 1, 0)
df_so['LV2_TocDoGiaoHang'] = np.where(df_so['1.1. Vận hành_CODING_SO'].str.contains(r'toc do', na = False), 1, 0)
df_so['LV2_TiLeHoanHang'] = np.where(df_so['1.1. Vận hành_CODING_SO'].str.contains(r'hoan hang cao', na = False), 1, 0)
df_so['LV2_KhongTrungThuc'] = np.where(df_so['1.1. Vận hành_CODING_SO'].str.contains(r'khong trung thuc', na = False), 1, 0)
df_so['LV2_MatHang'] = np.where(df_so['1.1. Vận hành_CODING_SO'].str.contains(r'mat hang', na = False), 1, 0)

df_so

In [ ]:
df_so_groupby = df_so.groupby(['THANG', 'CODING Tình trạng hiện tại']).agg(
                                    # GiamDon = ('is_giamdon', sum),
                                    # RoiBo = ('is_roibo', sum),
                                    SoLuong = ('CODING Tình trạng hiện tại', len),
                                    LV1_CaNhan = ('LV1_CaNhan', sum),
                                    LV1_VanHanh = ('LV1_VanHanh', sum),
                                    LV1_Gia = ('LV1_Gia', sum),
                                    LV1_DenBu = ('LV1_DenBu', sum),
                                    LV1_CSKH = ('LV1_CSKH', sum),
                                    LV1_Khac = ('LV1_Khac', sum),

                                    LV2_TatTuyen = ('LV2_TatTuyen', sum),
                                    LV2_TocDoGiaoHang = ('LV2_TocDoGiaoHang', sum),
                                    LV2_TiLeHoanHang = ('LV2_TiLeHoanHang', sum),
                                    LV2_KhongTrungThuc = ('LV2_KhongTrungThuc', sum),
                                    LV2_MatHang = ('LV2_MatHang', sum),

                                   )

In [ ]:
dividend_col_LV1 = ['LV1_VanHanh', 'LV1_Gia', 'LV1_DenBu', 'LV1_CSKH', 'LV1_Khac',]
PercentageColumn_LV1 = ['Percentage_LV1_VanHanh', 'Percentage_LV1_Gia', 'Percentage_LV1_DenBu', 'Percentage_LV1_CSKH', 'Percentage_LV1_Khac']

dividend_col_LV2 =     ['LV2_TatTuyen', 'LV2_TocDoGiaoHang', 'LV2_TiLeHoanHang', 'LV2_KhongTrungThuc', 'LV2_MatHang']
PercentageColumn_LV2 = ['Percentage_LV2_TatTuyen', 'Percentage_LV2_TocDoGiaoHang', 'Percentage_LV2_TiLeHoanHang', 'Percentage_LV2_KhongTrungThuc', 'Percentage_LV2_MatHang']

In [ ]:
df_so_groupby['Percentage_LV1_CaNhan'] = df_so_groupby['LV1_CaNhan'].div(df_so_groupby['SoLuong'], axis=0)
df_so_groupby[PercentageColumn_LV1] = df_so_groupby[dividend_col_LV1].div(df_so_groupby[dividend_col_LV1].sum(axis = 1), axis=0)
df_so_groupby[PercentageColumn_LV2] = df_so_groupby[dividend_col_LV2].div(df_so_groupby['LV1_VanHanh'], axis=0)

In [ ]:

df_so_groupby = df_so_groupby.reset_index()
df_so_groupby = pd.melt(df_so_groupby, id_vars = ['THANG', 'CODING Tình trạng hiện tại'],
                        value_vars = ['SoLuong','Percentage_LV1_CaNhan',
                                      'Percentage_LV1_VanHanh', 'Percentage_LV1_Gia','Percentage_LV1_DenBu', 'Percentage_LV1_CSKH', 'Percentage_LV1_Khac',
                                      'Percentage_LV2_TatTuyen', 'Percentage_LV2_TocDoGiaoHang', 'Percentage_LV2_TiLeHoanHang', 'Percentage_LV2_KhongTrungThuc', 'Percentage_LV2_MatHang'
                                      ], value_name = 'Value') # Melt

In [ ]:
df_so_groupby['HeaderName'] = df_so_groupby['variable'] + '_' + df_so_groupby['CODING Tình trạng hiện tại']
df_so_groupby

In [ ]:
df_so_groupby_1 = pd.pivot_table(df_so_groupby, values = 'Value', index = ['THANG'], columns = 'HeaderName').reset_index()
df_so_groupby_1

## **Retention A**

In [ ]:
wb_retention_a = gc.open_by_url('https://docs.google.com/spreadsheets/d/1hf0SEm5do8pr5xGEz-6cA2Y2IQOr7Ucnt9ccOYQEhXY/edit#gid=1749797888')
ws_retention_a = wb_retention_a.worksheet('data_all')
df_retention_a = pd.DataFrame(ws_retention_a.get_all_records())

In [ ]:
df_retention_a['Retention?'] = pd.to_numeric(df_retention_a['Retention?'], errors = 'coerce')
df_retention_a['Loai_Tru'] = pd.to_numeric(df_retention_a['Loai_Tru'], errors = 'coerce')

df_retention_a['Vung'] = df_retention_a['Vung'].astype(str).apply(no_accent_plus)
df_retention_a['department_name_2'] = np.where(df_retention_a['Vung'].isin(department_list), df_retention_a['Vung'], 'others')

cond = (df_retention_a['Tháng'] != '') & (df_retention_a['Loai_Tru'] ==  0) & (df_retention_a['Tháng'] >= '2023-09-01')
df_retention_a = df_retention_a[cond]
df_retention_a

In [ ]:
##### Toàn quốc

In [ ]:
fetch_col_retention_a = ['THANG', 'Percetage_RetentionA']

In [ ]:

df_retention_a_gb = df_retention_a.groupby('Tháng').agg(is_retention_A = ('Retention?', sum), TotalA = ('Tháng', len))
df_retention_a_gb['Percetage_RetentionA'] = df_retention_a_gb['is_retention_A']/df_retention_a_gb['TotalA']
df_retention_a_gb = df_retention_a_gb.reset_index().rename({'Tháng':'THANG'}, axis = 1)
df_retention_a_gb['THANG'] = df_retention_a_gb['THANG'].to_numpy().astype('datetime64[M]')
df_retention_a_gb['THANG'] = df_retention_a_gb['THANG'].dt.date

df_retention_a_gb = df_retention_a_gb[fetch_col_retention_a]
df_retention_a_gb

## **BigQuery Section**

### Số lượng KH A, KH ABCD (Chỉ theo Tháng)

In [ ]:
sql_crm = """SELECT  * FROM `ad_hoc.phan_nhom_kh_crm`
              WHERE THANG >= date '2023-09-01'
            """

In [ ]:
df_crm = gcp2df(sql_crm)
df_crm['Group'] = df_crm['nhom_thang_N'].str[0]
df_crm['is_khA'] = np.where(df_crm['Group'].str.match('A', na = False),1,0)
df_crm['TruHang_ThangHang'] = np.where(
                                        (df_crm['phan_loai_retention'].str.contains('Trụ hạng'))|
                                       (df_crm['phan_loai_retention'].str.contains('Lên hạng')),1,0

                                       )

df_crm['department_name'] = df_crm['department_name'].astype(str).apply(no_accent_plus)
df_crm['department_name_2'] = np.where(df_crm['department_name'].isin(department_list), df_crm['department_name'], 'others')


df_crm

In [ ]:
##### Toàn quốc

In [ ]:
df_crm_gb = df_crm.groupby(['THANG']).agg(TotalKH_ABCD = ('THANG', len), TotalKH_A = ('is_khA', sum), SLTruHang_ThangHang = ('TruHang_ThangHang', sum)).reset_index()
df_crm_gb['Percentage_RetentionABCD'] = df_crm_gb['SLTruHang_ThangHang']/df_crm_gb['TotalKH_ABCD']
fetch_col = ['THANG','TotalKH_ABCD', 'TotalKH_A', 'Percentage_RetentionABCD']
df_crm_gb = df_crm_gb[fetch_col]
df_crm_gb

### Số lượng KH Giảm đơn (Lùi về 1 Tuần)

In [ ]:
bucket_file = !gsutil ls gs://cx_ad_hoc/Client_Declining_Revenue/
bucket_list = [x for x in bucket_file if x != 'gs://cx_ad_hoc/Client_Declining_Revenue/']
bucket_list

In [ ]:
file_name = 'data.csv'
df_planning = pd.DataFrame()
for root in bucket_list:
  file_name_final = root + file_name
  df_frag = pd.read_csv(file_name_final)
  Dot = root.split('/')[-2].strip('Dot=')
  df_frag = df_frag.replace('',np.nan)
  df_frag['Đợt'] = Dot

  df_frag['Dot_V2'] = pd.to_datetime(df_frag['Đợt']) - timedelta(weeks = 1)
  df_frag['THANG'] = df_frag['Dot_V2'].to_numpy().astype('datetime64[M]')
  df_frag['THANG'] = df_frag['THANG'].dt.date

  df_frag['Dot_T3'] = df_frag['Dot_V2'].astype(str).apply(WeekdayCalculation)
  df_frag['WeekFinal_2'] = df_frag['Dot_V2'].dt.strftime('%Y-%m-%d').apply(WeekRangeInput)


  df_frag['department_name'] = df_frag['department_name'].astype(str).apply(no_accent_plus)
  df_frag['department_name_2'] = np.where(df_frag['department_name'].isin(department_list), df_frag['department_name'], 'others')


  df_frag['clientid'] = df_frag['clientid'].astype(str)
  df_frag['sale_owner_id'] = df_frag['sale_owner_id'].map(str)
  df_frag['section_manager_id'] = df_frag['section_manager_id'].map(str)

  df_frag = df_frag.replace(r'\.0$', '', regex=True)
  df_planning = pd.concat([df_frag, df_planning])
df_planning

In [ ]:
cond = (df_planning['nhom_phan_tram_giam'] == '>30') & (df_planning['THANG'] >= pd.to_datetime('2023-09-01') )
df_planning = df_planning[cond]
df_planning

In [ ]:
col_list = ['THANG','Dot_V2', 'department_name_2','WeekFinal_2', 'clientid', 'nhom']
df_planning = df_planning[col_list]
df_planning

#### Theo Tháng

In [ ]:
df_planning_gb = df_planning.groupby('THANG').agg(SLKHGiamDonTren30 = ('clientid', len), SLKHGiamDonTren30_Unique = ('clientid', 'nunique') ).reset_index()
df_planning_gb

### **Retention Revenue**

#### Theo Tháng

In [ ]:
sql_revenue = """
                WITH
                MY_TEMP_TOTAL AS (
                SELECT Month, SUM(Revenue) AS TOTAL_REVENUE FROM `ad_hoc.data_retention_daily` as L
                GROUP BY MONTH
                ),
                MY_TEMP_AF AS (
                SELECT Month, SUM(Revenue) AS REVENUE_AF FROM `ad_hoc.data_retention_daily` as L
                where L.Group != 'G' and L.Group != 'H'
                Group by Month
                )
              SELECT L.MONTH AS THANG, L.REVENUE_AF/R.TOTAL_REVENUE AS Percentage_RetentionRevenue FROM MY_TEMP_AF AS L
              LEFT JOIN MY_TEMP_TOTAL AS R
              ON L.MONTH = DATE_ADD(R.MONTH, INTERVAL 1 MONTH)
              WHERE L.MONTH >= DATE '2023-09-01'
              ORDER BY L.MONTH DESC
              """

sql_revenue_department = """
            WITH ORIGINAL_TABLE AS(
                SELECT *,
                (CASE WHEN department_name in ('BD NORTH', 'BD SOUTH', 'HNO', 'DSH')
                THEN lower(department_name)
                ELSE 'others'
                END
                ) as department_name_2
                FROM `ad_hoc.data_retention_daily`

            ),
            MY_TEMP_TOTAL AS (
            SELECT Month, department_name_2,SUM(Revenue) AS TOTAL_REVENUE FROM ORIGINAL_TABLE as L
            GROUP BY MONTH, department_name_2
            ),
            MY_TEMP_AF AS (
            SELECT Month, department_name_2, SUM(Revenue) AS REVENUE_AF FROM ORIGINAL_TABLE as L
            where L.Group != 'G' and L.Group != 'H'
            Group by Month, department_name_2
            )

            SELECT L.MONTH AS THANG, L.department_name_2, L.REVENUE_AF/R.TOTAL_REVENUE AS Percentage_RetentionRevenue FROM MY_TEMP_AF AS L
            LEFT JOIN MY_TEMP_TOTAL AS R
            ON L.MONTH = DATE_ADD(R.MONTH, INTERVAL 1 MONTH)
            AND L.department_name_2 = R.department_name_2

            WHERE L.MONTH >= DATE '2023-09-01'
            ORDER BY L.MONTH DESC

            """

In [ ]:
df_retention_revenue_gb = gcp2df(sql_revenue)
df_retention_revenue_gb

## **Merge Dataframes**

In [ ]:
data_frames = [df_so_groupby_1, df_crm_gb, df_planning_gb, df_retention_revenue_gb, df_retention_a_gb]
print(len(data_frames))
df_merged = reduce(lambda left, right: pd.merge(left, right, on = ['THANG'], how = 'outer'), data_frames)
df_merged

In [ ]:
df_merged = pd.pivot_table(df_merged, columns = 'THANG').reset_index()
df_merged

# **Bảng 3**

In [ ]:
wb_call = gc.open_by_url('https://docs.google.com/spreadsheets/d/1WsHPDX4Pms1TAzAZrlt7OfZUlFd1lM5EoXAdJfvCEZw/edit#gid=1595060363')
ws_call = wb_call.worksheet('QC Call KH')

In [ ]:
df_call_raw = pd.DataFrame(ws_call.get_all_records())
df_call_raw

In [ ]:
fetch_col = [
            'Đợt', 'ID KH', 'Nhóm KH đang thăm hỏi', 'department_name',
            'Level 1_CODING_CS', '1.1. Vận hành_CODING_CS',
            'Share đơn ĐVVC khác\n(1: có, 0: không)_KH',
            'Q4\n(Optional)_(1) Mức độ hài lòng nói chung về dịch vụ :\n1. Rất tệ/ rất không hài lòng ... tăng dần đến ....10. Rất tốt/ rất hài lòng',
            'Q3_2 tuần gần đây, Anh/ chị thấy tốc độ giao hàng  như thế nào so với nhu cầu của shop anh/ chị? Vui lòng đánh giá theo thang điểm 10 với:\n1 điểm - Rất chậm, không thể chấp nhận được tăng dần đến 10 điểm -Tốc độ nhanh xuất sắc',
            'Q4\n(Optional)_(2) Đánh giá dịch vụ chăm sóc khách hàng: 1. Rất không nhiệt tình ....tăng dần đến ....10. Rất nhiệt tình ',
            'Q4\n(Optional)_(4) Đánh giá thái độ nhân viên giao hàng: 1. Rất tệ ....tăng dần đến ....10. Rất tốt',
              ]

In [ ]:
rename_col = ['Dot', 'ClientID', 'Group', 'department_name',
              'Level 1_CODING_CS','1.1. Vận hành_CODING_CS',
              'is_sharedon',
              'DanhGiaChungDichVu',
              'DanhGiaTocDoGiaoHang',
              'DanhGiaCSKH',
              'DanhGiaShipper'
              ]

numeric_col = ['is_sharedon', 'DanhGiaTocDoGiaoHang', 'DanhGiaChungDichVu', 'DanhGiaCSKH', 'DanhGiaShipper']

In [ ]:
df_call = df_call_raw[fetch_col]
df_call.columns = rename_col

In [ ]:
df_call['Dot'] = pd.to_datetime(df_call['Dot'], errors = 'coerce')
df_call['Dot_V2'] = df_call['Dot'] #- timedelta(weeks = 1)
df_call['WeekFinal_2'] = df_call['Dot_V2'].dt.strftime('%Y-%m-%d').apply(WeekRangeInput)
df_call = df_call[df_call['Dot_V2'] >= '2023-09-01']

df_call['THANG'] = df_call['Dot_V2'].to_numpy().astype('datetime64[M]')
df_call['THANG'] = df_call['THANG'].dt.date

df_call['department_name'] = df_call['department_name'].astype(str).apply(no_accent_plus)
df_call['department_name_2'] = np.where(df_call['department_name'].isin(department_list), df_call['department_name'], 'others')

df_call['Dot_T3'] = df_call['Dot_V2'].astype(str).apply(WeekdayCalculation)
df_call['id_col'] = df_call['ClientID'].astype(str) + '_' +df_call['Dot_T3'] #Dùng để drop Duplicates

df_call['Level 1_CODING_CS'] = df_call['Level 1_CODING_CS'].astype(str).apply(no_accent_plus)
df_call['1.1. Vận hành_CODING_CS'] = df_call['1.1. Vận hành_CODING_CS'].astype(str).apply(no_accent_plus)

df_call = df_call.replace(r'^\s*$', np.nan, regex=True)
df_call[numeric_col] = df_call[numeric_col].astype(float)


In [ ]:
df_call['LV1_CaNhan'] = np.where(df_call['Level 1_CODING_CS'].str.contains(r'ca nhan', na = False), 1, 0)
df_call['is_coding_LV1'] = np.where(df_call['Level 1_CODING_CS'].isna(), 0, 1)

df_call['LV1_VanHanh'] = np.where(df_call['Level 1_CODING_CS'].str.contains(r'van hanh', na = False), 1, 0)
df_call['LV1_Gia'] = np.where(df_call['Level 1_CODING_CS'].str.contains(r'gia|phu phi', na = False), 1, 0)
df_call['LV1_DenBu'] = np.where(df_call['Level 1_CODING_CS'].str.contains(r'den bu', na = False), 1, 0)
df_call['LV1_CSKH'] = np.where(df_call['Level 1_CODING_CS'].str.contains(r'cham soc khach hang|cskh', na = False), 1, 0)

df_call['LV2_TatTuyen'] = np.where(df_call['1.1. Vận hành_CODING_CS'].str.contains(r'tat tuyen', na = False), 1, 0)
df_call['LV2_TocDoGiaoHang'] = np.where(df_call['1.1. Vận hành_CODING_CS'].str.contains(r'toc do', na = False), 1, 0)
df_call['LV2_TyLeHoanHang'] = np.where(df_call['1.1. Vận hành_CODING_CS'].str.contains(r'hoan hang cao', na = False), 1, 0)
df_call['LV2_KhongTrungThuc'] = np.where(df_call['1.1. Vận hành_CODING_CS'].str.contains(r'khong trung thuc', na = False), 1, 0)
df_call['LV2_MatHang'] = np.where(df_call['1.1. Vận hành_CODING_CS'].str.contains(r'mat hang', na = False), 1, 0)

df_call['is_CSAT_TocDoGiao'] = np.where(df_call['DanhGiaTocDoGiaoHang'].isna(), 0, 1)
df_call['T2B_CSAT_TocDoGiao'] = np.where(df_call['DanhGiaTocDoGiaoHang'] >= 4, 1, 0)
df_call['TB_CSAT_TocDoGiao'] = np.where(df_call['DanhGiaTocDoGiaoHang'] == 5, 1, 0)

df_call['is_CSAT_CSKH'] = np.where(df_call['DanhGiaCSKH'].isna(), 0, 1)
df_call['T2B_CSAT_CSKH'] = np.where(df_call['DanhGiaCSKH'] >= 4, 1, 0)
df_call['TB_CSAT_CSKH'] = np.where(df_call['DanhGiaCSKH'] == 5, 1, 0)

df_call['is_CSAT_Shipper'] = np.where(df_call['DanhGiaShipper'].isna(), 0, 1)
df_call['T2B_CSAT_Shipper'] = np.where(df_call['DanhGiaShipper'] >= 4, 1, 0)
df_call['TB_CSAT_Shipper'] = np.where(df_call['DanhGiaShipper'] == 5, 1, 0)

In [ ]:
df_call_gb = df_call.groupby('THANG').agg(
                              SLKHCallThanhCong = ('ClientID', len),
                              is_coding_LV1 = ('is_coding_LV1', sum),
                              LV1_CaNhan = ('LV1_CaNhan', sum),

                              LV1_VanHanh = ('LV1_VanHanh', sum),
                              LV1_Gia = ('LV1_Gia', sum),
                              LV1_DenBu = ('LV1_DenBu', sum),
                              LV1_CSKH = ('LV1_CSKH', sum),

                              LV2_TatTuyen = ('LV2_TatTuyen', sum),
                              LV2_TocDoGiaoHang = ('LV2_TocDoGiaoHang', sum),
                              LV2_TyLeHoanHang = ('LV2_TyLeHoanHang', sum),
                              LV2_KhongTrungThuc = ('LV2_KhongTrungThuc', sum),
                              LV2_MatHang = ('LV2_MatHang', sum),

                              SLKHShareDon = ('is_sharedon', sum),

                              is_CSAT_TocDoGiao = ('is_CSAT_TocDoGiao', sum),
                              T2B_CSAT_TocDoGiao = ('T2B_CSAT_TocDoGiao', sum),
                              TB_CSAT_TocDoGiao = ('TB_CSAT_TocDoGiao', sum),

                              is_CSAT_CSKH = ('is_CSAT_CSKH', sum),
                              T2B_CSAT_CSKH = ('T2B_CSAT_CSKH', sum),
                              TB_CSAT_CSKH = ('TB_CSAT_CSKH', sum),

                              is_CSAT_Shipper = ('is_CSAT_Shipper', sum),
                              T2B_CSAT_Shipper = ('T2B_CSAT_Shipper', sum),
                              TB_CSAT_Shipper = ('TB_CSAT_Shipper', sum),
                              )





In [ ]:
sum_4_cols = ['LV1_VanHanh'	,'LV1_Gia'	,'LV1_DenBu'	,'LV1_CSKH']

self_divide_cols = ['T2B_CSAT_TocDoGiao', 'TB_CSAT_TocDoGiao',
                    'T2B_CSAT_CSKH', 'TB_CSAT_CSKH',
                    'T2B_CSAT_Shipper', 'TB_CSAT_Shipper'
                    ]

In [ ]:
df_call_gb['sum_4_col'] = df_call_gb[sum_4_cols].sum(axis = 1)

df_call_gb

In [ ]:
pattern = re.compile(r'LV2')

for name in df_call_gb.iloc[:,1:].columns:
  new_name = 'Percentage_Call_' + name
  if re.match(pattern, name):
    #print(name)
    df_call_gb[new_name] = df_call_gb[name].div(df_call_gb['LV1_VanHanh'], axis = 0)
  elif name in sum_4_cols:
    df_call_gb[new_name] = df_call_gb[name].div(df_call_gb['sum_4_col'], axis = 0)
  elif name == 'LV1_CaNhan':
    df_call_gb[new_name] = df_call_gb[name].div(df_call_gb['is_coding_LV1'], axis = 0)
  elif name in self_divide_cols:
    divisor_name = 'is_' + name.split('_', 1)[1]
    df_call_gb[new_name] = df_call_gb[name].div(df_call_gb[divisor_name], axis = 0)
  else:
    df_call_gb[new_name] = df_call_gb[name].div(df_call_gb['SLKHCallThanhCong'], axis = 0)
df_call_gb

In [ ]:
fetch_export_col = ['SLKHCallThanhCong',
                    'Percentage_Call_LV1_CaNhan',
                    'Percentage_Call_LV1_VanHanh','Percentage_Call_LV1_Gia','Percentage_Call_LV1_DenBu', 'Percentage_Call_LV1_CSKH',
                    "Percentage_Call_LV2_TatTuyen","Percentage_Call_LV2_TocDoGiaoHang","Percentage_Call_LV2_TyLeHoanHang","Percentage_Call_LV2_KhongTrungThuc","Percentage_Call_LV2_MatHang",

                    'Percentage_Call_SLKHShareDon',
                    'Percentage_Call_T2B_CSAT_TocDoGiao', 'Percentage_Call_TB_CSAT_TocDoGiao',
                    'Percentage_Call_T2B_CSAT_CSKH','Percentage_Call_TB_CSAT_CSKH',
                    'Percentage_Call_T2B_CSAT_Shipper','Percentage_Call_TB_CSAT_Shipper'
                    ]

In [ ]:
df_call_gb_final = df_call_gb[fetch_export_col].T.reset_index()
df_call_gb_final

# **Push to GGSheets**

In [ ]:
department_list = ['BD NORTH', 'BD SOUTH', 'DSH', 'HNO', 'OTHERS']

In [ ]:
ws_dict = gc.open_by_url('https://docs.google.com/spreadsheets/d/1YHo8AHtfQf3tpmhXl5Dsxc4Rf1nJRM5zHHK71Gaa14U/edit#gid=997727404').worksheet('dict')
df_dict = pd.DataFrame(ws_dict.get_all_records())
zip_dict = dict(zip(df_dict['index'], df_dict['NameFinal']))
zip_dict

In [ ]:
df_merged_2 = df_merged.replace(zip_dict)
df_merged_2

In [ ]:
df_merged_2 = df_merged_2.merge(df_dict, how = 'left', left_on = 'index',right_on = 'NameFinal').sort_values(by = 'Order_TQ_Month', ascending = True)\
                          .drop(['Order', 'Order_TQ_Week', 'Order_TQ_Month', 'NameFinal', 'index_y'], axis = 1)
df_merged_2

In [ ]:
df_call_gb_final_2 = df_call_gb_final.replace(zip_dict)
df_call_gb_final_2

In [ ]:
df_call_gb_final_2 = df_call_gb_final_2.merge(df_dict, how = 'left', left_on = 'index',right_on = 'NameFinal').sort_values(by = 'Order_TQ_Month', ascending = True)\
                                        .drop(['Order', 'Order_TQ_Week', 'Order_TQ_Month', 'NameFinal', 'index_y'], axis = 1)
df_call_gb_final_2

In [ ]:
sheet_name = 'Toàn quốc'
wb_export = gc.open_by_url('https://docs.google.com/spreadsheets/d/1H-jZBIfhIKMDhIYRwSgfxWx1hmdSOY20Pu_gBonb2WI/edit#gid=0')
ws_export = wb_export.worksheet(sheet_name)
ws_export.clear()

set_with_dataframe(ws_export, df_merged_2, col = 11)
set_with_dataframe(ws_export, df_call_gb_final_2, row = 40, col = 11)

### Sheet MTD

In [ ]:
pd.set_option('display.max_rows', 50)

In [ ]:
df_merged['department_name_2'] = 'Toàn quốc'
df_call_gb_final['department_name_2'] = 'Toàn quốc'

In [ ]:
list_df = [df_merged, df_call_gb_final, df_department_merged, df_call_department_gb_final]

In [ ]:
df_merged_mtd = pd.concat(list_df).astype(str)
df_merged_mtd

In [ ]:
month_value = '2024-01-01'

In [ ]:
df_merged_mtd = pd.melt(df_merged_mtd, id_vars = ['index', 'department_name_2'])
df_merged_mtd['value'] = pd.to_numeric(df_merged_mtd['value'], errors = 'coerce')
# df_merged_mtd['value'] = df_merged_mtd['value'].fillna('')
df_merged_mtd

In [ ]:
df_mtd_export = df_merged_mtd[df_merged_mtd['THANG'] == pd.to_datetime(month_value)]
df_mtd_export = pd.pivot_table(df_mtd_export, index = 'index', columns = 'department_name_2').droplevel(0, axis = 1).reset_index()
df_mtd_export.fillna('', inplace = True)
df_mtd_export

In [ ]:
df_dict = df_dict[df_dict['Order'] != 999][['index', 'Order']]
df_dict

In [ ]:
df_mtd_export = df_mtd_export.merge(df_dict, how = 'left').sort_values(by = 'Order', ascending = True).drop('Order', axis = 1)
df_mtd_export

In [ ]:
sheet_name = 'MTD'
wb_export = gc.open_by_url('https://docs.google.com/spreadsheets/d/1H-jZBIfhIKMDhIYRwSgfxWx1hmdSOY20Pu_gBonb2WI/edit#gid=0')
ws_export = wb_export.worksheet(sheet_name)
ws_export.clear()
set_with_dataframe(ws_export, df_mtd_export.replace(zip_dict), col = 5)
ws_export.update('A1', month_value)